In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Traning of a Logistic Regression model with scikit-learn to predict weather

### steps 
* Exploratory data analysis and visualization
* Spiling the dataset into traning,validation and test sets
* Impution of mission values 
* Encoding categorical data as one-hot vectors
* Traning a logistic regression model 
* Evaluting a model using a validation set and test set

### Objective

The used dataset contains about ten years of daily weather observation from different  Australian Weather Station. 

We train a model to create a system that can use today's weather data for a given location predict whether it will rain at that location tomorrow or not. 

#### Let's install scikit-learn library

In [ ]:
!pip install scikit-learn 

In [ ]:
path_name = '/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv'

In [ ]:
import pandas as pd
weather_df = pd.read_csv(path_name)

In [ ]:
weather_df

The dataset contain numeric , categorical and object columns. Our objective is to create a model to predict the data in RainTomorrow column.

### Let's look into the missing values

In [ ]:
weather_df.info()

It might be helpful to remove the raws with missing values in RainToday and RainTomorrow columns to make our analysis and modeling simplier as one of them will be our target and other is likely to be closely related to the target variable.

In [ ]:
weather_df.dropna(subset = ['RainToday' ,'RainTomorrow'] ,inplace = True)

## Exploratory Data analysis and visualization 

Before train our machine learning model we will be doing some analysis of the columns and visualize them and see how they are related to our target.

#### Let's install the  packages / libraries

In [ ]:
!pip install plotly matplotlib seaborn

In [ ]:
import plotly.express as px
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['figure.figsize'] = (10 , 6)

In [ ]:
px.histogram(weather_df , x = 'Location' , title ='Location vs. Rainy Days' ,color ='RainToday')

In [ ]:
weather_df['Humidity3pm'].isna().sum()

In [ ]:
px.histogram(weather_df , x = 'Humidity3pm' , title ='Humidity at 3pm vs. Rain Tomorrow' ,color ='RainTomorrow')

Humidity at 3 pm doesn't have any clear relation with Rain Tomorrow. 

In [ ]:
px.histogram(weather_df , x = 'RainTomorrow' , 
             title = 'Rain Tomorrow vs Rain Today',
            color = 'RainToday')

In [ ]:
px.scatter(weather_df.sample(3000), 
           title='Temp (3 pm) vs. Humidity (3 pm)',
           x='Temp3pm',
           y='Humidity3pm',
           color='RainTomorrow')

In [ ]:
px.scatter(weather_df.sample(3000), 
           title='Min Temp. vs Max Temp.',
           x='MinTemp', 
           y='MaxTemp', 
           color='RainTomorrow')

### Traing , Validation and Test set

In [ ]:
weather_df

As Date column is given ,so we will be separating traning , validation and test sets with time. So that the model is trained on data from the past and evaluated on data from the future. 

For the current dataset ,we can use Date column to create a Year column in the dataset.

In [ ]:
Year = pd.to_datetime(weather_df.Date).dt.year

In [ ]:
plt.title('No of rows per year')
sns.countplot(x= Year);

We'll pick last two years for test set and an year before it for validation set.

In [ ]:
train_df = weather_df[Year < 2015]
val_df = weather_df[Year == 2015]
test_df = weather_df[Year > 2015]

print('train_df.shape :', train_df.shape)
print('val_df.shape :', val_df.shape)
print('test_df.shape :', test_df.shape)

In [ ]:
train_df

In [ ]:
val_df

In [ ]:
test_df

### Input and Target column.

In [ ]:
input_col = list(weather_df.columns)[1:-1]
tar_col = 'RainTomorrow'

In [ ]:
print(input_col)

In [ ]:
print(tar_col)

In [ ]:
train_input = train_df[input_col].copy()
train_tar = train_df[tar_col].copy()

In [ ]:
train_input

In [ ]:
train_tar

In [ ]:
val_input = val_df[input_col].copy()
val_tar = val_df[tar_col].copy()

In [ ]:
test_input = test_df[input_col].copy()
test_tar = test_df[tar_col].copy()

In [ ]:
!pip install numpy

In [ ]:
import numpy as np

In [ ]:
numeric_cols = train_input.select_dtypes(include = np.number).columns.tolist()[:-1]
categorical_cols = train_input.select_dtypes('object').columns.tolist()

In [ ]:
train_input[categorical_cols].nunique()

In [ ]:
list(train_input[numeric_cols])

### Imputing  Missing numeric data


There are various techniques in machine learning for imputation of missing value . Here we will use one. We will replace the missing data with the average value of the column using SimpleImputer class.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy = 'mean')

In [ ]:
imputer.fit(weather_df[numeric_cols])

In [ ]:
list(imputer.statistics_)

In [ ]:
train_input[numeric_cols] = imputer.transform(train_input[numeric_cols])
val_input[numeric_cols] = imputer.transform(val_input[numeric_cols])
test_input[numeric_cols] = imputer.transform(test_input[numeric_cols])

In [ ]:
train_input[numeric_cols].describe()

### Encoding Categorical Data

Machine learning models can only be trained with numeric data, so we need to convert categorical data into numeric . Here we will use OneHotEncoding method.One hot encoding involves adding a new binary (0/1) column for each unique category of a categorical column.

In [ ]:
weather_df[categorical_cols].nunique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
?OneHotEncoder

In [ ]:
encoder = OneHotEncoder(sparse=False, handle_unknown ='ignore')

In [ ]:
weather_df2 = weather_df[categorical_cols].fillna('Unknown')

In [ ]:
encoder.fit(weather_df2)

In [ ]:
encoder.categories_

The encoder has created a list of categories for each of the categorical columns in the dataset. 

We can generate column names for each individual category using `get_feature_names`.

In [ ]:
encoded_cols = list(encoder.get_feature_names(categorical_cols))
print(encoded_cols)

In [ ]:
train_input[encoded_cols] = encoder.transform((train_input[categorical_cols]).fillna('Unknown'))
val_input[encoded_cols] = encoder.transform((val_input[categorical_cols]).fillna('Unknown'))
test_input[encoded_cols] = encoder.transform((test_input[categorical_cols]).fillna('Unknown'))

In [ ]:
test_input

## Training the logistic Regression Model

To train a logistic regression model, we can use the LogisticRegression class from Scikit-learn.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver = 'liblinear')

In [ ]:
model.fit(train_input[numeric_cols + encoded_cols] , train_tar)

In [ ]:
print(model.coef_.tolist())

In [ ]:
print(model.intercept_)

## Making Predictions and evaluating the model

In [ ]:
X_train = train_input[numeric_cols + encoded_cols]
X_val = val_input[numeric_cols + encoded_cols]
X_test = test_input[numeric_cols + encoded_cols]

In [ ]:
train_preds = model.predict(X_train)

In [ ]:
train_preds

In [ ]:
train_tar

We can output a probabilistic prediction using `predict_proba`.

In [ ]:
train_probs = model.predict_proba(X_train)
train_probs

The numbers above indicate the probabilities for the target classes "No" and "Yes".

In [ ]:
model.classes_

We can test the accuracy of the model's predictions by computing the percentage of matching values in `train_preds` and `train_targets`.

This can be done using the `accuracy_score` function from `sklearn.metrics`.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(train_tar, train_preds)

The model accuracy is shown about 84%.